# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

* Student Name : Radhika Balasubramaniam
* Project Description : Working with weather data to plan future vacations using jupyter-gmaps and the Google Places API 




# Program implementation starts here

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# read the csv file 

output_path = os.path.join("..","Resources", "cities.csv")
citiesDF = pd.read_csv(output_path, encoding="utf-8")
citiesDF.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.80,-139.03,77.52,77,40,22.37,PF,1600357383
1,Punta Arenas,-53.15,-70.92,41.00,73,40,38.03,CL,1600357183
2,Klaksvík,62.23,-6.59,53.60,100,90,6.93,FO,1600357385
3,Ushuaia,-54.80,-68.30,35.60,64,75,38.03,AR,1600357261
4,Lorengau,-2.02,147.27,81.23,81,95,9.71,PG,1600357386


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)



In [19]:
# location co-ordinates for the cities
coordinates = citiesDF[['Lat' ,'Lng']]
humidity = citiesDF['Humidity']


# create the mapping figure
fig = gmaps.figure()

# create the heatmap layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity)

# add the heatmap layer to the figure
fig.add_layer(heat_layer)



### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:

# get the cities with ideal temperatures for vacation
idealWeatherCitiesDF = citiesDF[(citiesDF['Max Temp']>= 60) & (citiesDF['Max Temp'] <= 80)]

# drop any rows with null values
idealWeatherCitiesDF.dropna(how='any')

# reset the index of the idealWeatherCities
idealWeatherCitiesDF = idealWeatherCities.reset_index(drop=True)



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# create a hotel list 
hotel_df = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# distance, IN METERS, within which the place results must live
target_radius = 5000

# type of establishment that we'd like to use to filter our place results
target_type = "hotel"

#loop through cities list and build the hotel list for the ideal weather cities
for  index, row in idealWeatherCitiesDF.iterrows() :
   
    try :      
        
        #set up target location
        target_location = f"{row['Lat']},{row['Lng']}"
       
        # set up a dictionary to store all query parameters
        
        params = {
            "location": target_location,     
            "radius": target_radius,
            "type": target_type,
            "key": g_key
            }
        
        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        
        try : 
            
            # convert the response to json
            response_json = response.json()
            
            try :
                # retrieve the hotel name, if not available make it empty as hotel name for the city
                hotelName = response_json['results'][0]['name']
                
            except  Exception as hEx:
                #exception while reading hotel name from response
                
                print(f"data retrieval exception : {hEx}")
                hotelName = None
            
            # append the hotel to hotel_df
            hotel_df.append(hotelName)
            
            
        except Exception as rEx:
            #exception while converting response to json
            
            print(f"response parse exception : {rEx}")            
            hotel_df.append(None)
            
    except Exception as aEx:
        # exception while API query 
        
        print(f"api parse exception : {aEx}")                
        hotel_df.append(None)
    
# ***** for loop completed


# append the hotel list for the idealcitiesDF
idealWeatherCitiesDF['Hotel Name'] = hotel_df

idealWeatherCitiesDF

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Atuona,-9.80,-139.03,77.52,77,40,22.37,PF,1600357383,Atuona
1,Vaini,-21.20,-175.20,71.60,78,75,11.41,TO,1600357141,Nukuhetulu
2,Hilo,19.73,-155.09,71.60,78,1,3.36,US,1600357170,Hilo
3,San Isidro,-12.12,-77.05,62.60,77,90,4.70,PE,1600357388,Lima
4,Noumea,-22.28,166.46,64.40,88,21,2.24,NC,1600357374,Noumea
...,...,...,...,...,...,...,...,...,...,...
235,Kasungu,-13.03,33.48,79.88,29,39,8.43,MW,1600357605,Kasungu
236,Freeport,40.66,-73.58,75.00,64,20,9.17,US,1600357421,Hempstead
237,Walvis Bay,-22.96,14.51,68.00,45,0,9.17,NaN,1600357605,Walvis Bay
238,Mendoza,-32.89,-68.83,73.40,20,0,5.82,AR,1600357606,Mendoza


In [46]:
idealWeatherCitiesDF.dropna(subset = ["Hotel Name"])
idealWeatherCitiesDF

 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Atuona,-9.80,-139.03,77.52,77,40,22.37,PF,1600357383,Atuona
1,Vaini,-21.20,-175.20,71.60,78,75,11.41,TO,1600357141,Nukuhetulu
2,Hilo,19.73,-155.09,71.60,78,1,3.36,US,1600357170,Hilo
3,San Isidro,-12.12,-77.05,62.60,77,90,4.70,PE,1600357388,Lima
4,Noumea,-22.28,166.46,64.40,88,21,2.24,NC,1600357374,Noumea
...,...,...,...,...,...,...,...,...,...,...
235,Kasungu,-13.03,33.48,79.88,29,39,8.43,MW,1600357605,Kasungu
236,Freeport,40.66,-73.58,75.00,64,20,9.17,US,1600357421,Hempstead
237,Walvis Bay,-22.96,14.51,68.00,45,0,9.17,NaN,1600357605,Walvis Bay
238,Mendoza,-32.89,-68.83,73.40,20,0,5.82,AR,1600357606,Mendoza


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in idealWeatherCitiesDF.iterrows()]
locations = idealWeatherCitiesDF[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)


# Display figure 
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# add customizations to the map 

# Dictionary of style settings for the map
figure_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '3px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# create a new figure using the custom styles above
cust_fig = gmaps.figure(layout=figure_layout)

# Add the marker layer using the coordinates
cust_fig.add_layer(markers)

# Display the figure
cust_fig


Figure(layout=FigureLayout(border='3px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…